<a href="https://colab.research.google.com/github/olsenkat/ProgrammingExamChess/blob/main/embed/VectorDB_Lab_CS452_(starter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Download datasets from kaggle

import json
import os

if not os.path.exists("lex-fridman-text-embedding-3-large-128.zip"):
  kaggle_json = {"username": "michaeltreynolds","key": "149701be742f30a8a0526762c61beea0"}
  kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
  os.makedirs(kaggle_dir, exist_ok=True)
  kaggle_config_path = os.path.join(kaggle_dir, "kaggle.json")
  with open(kaggle_config_path, 'w') as f:
    json.dump(kaggle_json, f)

  !kaggle datasets download -d michaeltreynolds/lex-fridman-text-embedding-3-large-128


In [9]:
# Unzip kaggle data

!unzip lex-fridman-text-embedding-3-large-128.zip
!unzip lex-fridman-text-embedding-3-large-128/*.zip


Archive:  lex-fridman-text-embedding-3-large-128.zip
replace documents/documents/batch_request_0lw3vrQqdWbdBRurTGNMHU76.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: documents/documents/batch_request_0lw3vrQqdWbdBRurTGNMHU76.jsonl  
  inflating: documents/documents/batch_request_3GozevpriRRzieX4za9xfNmY.jsonl  
  inflating: documents/documents/batch_request_3maYxwEF1svWRpbYr10br6Wv.jsonl  
  inflating: documents/documents/batch_request_7hQA9m3pUXx22JXInjYZNAu2.jsonl  
  inflating: documents/documents/batch_request_7oR3UbWsHgESFLr5eqL3jkMD.jsonl  
  inflating: documents/documents/batch_request_CXU6VbN4SDinplgj1MpILc1u.jsonl  
  inflating: documents/documents/batch_request_Fve0NjohSf5Qe0bZtAnp8D5A.jsonl  
  inflating: documents/documents/batch_request_If8QibqlgU0XRU5FcD5zpiuL.jsonl  
  inflating: documents/documents/batch_request_KQAdZBdQHYMI2MfEMXf3ytLM.jsonl  
  inflating: documents/documents/batch_request_MFMHpnaCSkNrbgAgOxCzaLOl.jsonl  
  inflating: documents/documents/

In [3]:
# Use specific libraries
!pip install datasets==2.20.0 psycopg2==2.9.9 pgcopy==1.6.0
import psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.6 MB/s eta 0:00:00
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp312-cp312-linux_x86_64.whl size=533259 sha256=98f253fd2cd286f5414c59238a61c14936c57e8f43589820777a1e0250279993
  Stored in directory: /root/.cache/pip/wheels/ff/ac/80/7ccec163e3d05ae2112311b895132409b9abfd7e0c1c6b5183
Successfully built psycopg2
  Attempting uninstall: psycopg2
    Found existing installation: psycopg2 2.9.10
    Uninstalling psycopg2-2.9.10:
      Successfully uninstalled psycopg2-2.9.10
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Unins

In [ ]:
# Get your own trial account at timescaledb and paste your own connection string

#TODO
CONNECTION = "<YOUR CONNECTION>"

In [ ]:
# Use this if you want to start over on your postgres table!

DROP_TABLE = "DROP TABLE IF EXISTS podcast, segment"
with psycopg2.connect(CONNECTION) as conn:
    cursor = conn.cursor()
    cursor.execute(DROP_TABLE)
    conn.commit() # Commit the changes


In [ ]:
# Useful function that takes a pd.DataFrame and copies it directly into a table.

import pandas as pd
import io
import psycopg2

from typing import List

def fast_pg_insert(df: pd.DataFrame, connection: str, table_name: str, columns: List[str]) -> None:
    """
        Inserts data from a pandas DataFrame into a PostgreSQL table using the COPY command for fast insertion.

        Parameters:
        df (pd.DataFrame): The DataFrame containing the data to be inserted.
        connection (str): The connection string to the PostgreSQL database.
        table_name (str): The name of the target table in the PostgreSQL database.
        columns (List[str]): A list of column names in the target table that correspond to the DataFrame columns.

        Returns:
        None
    """
    conn = psycopg2.connect(connection)
    _buffer = io.StringIO()
    df.to_csv(_buffer, sep=";", index=False, header=False)
    _buffer.seek(0)
    with conn.cursor() as c:
        c.copy_from(
            file=_buffer,
            table=table_name,
            sep=";",
            columns=columns,
            null=''
        )
    conn.commit()
    conn.close()

Database Schema
We will create a database with two tables: podcast and segment:

**podcast**

- PK: id
 - The unique podcast id found in the huggingface data (i,e., TRdL6ZzWBS0  is the ID for Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214)
- title
 - The title of podcast (ie., Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214)

**segment**

- PK: id
 - the unique identifier for the podcast segment. This was created by concatenating the podcast idx and the segment index together (ie., "0;1") is the 0th podcast and the 1st segment
This is present in the as the "custom_id" field in the `embedding.jsonl` and batch_request.jsonl files
- start_time
 - The start timestamp of the segment
- end_time
 - The end timestamp of the segment
- content
 - The raw text transcription of the podcast
- embedding
 - the 128 dimensional vector representation of the text
- FK: podcast_id
 - foreign key to podcast.id

In [ ]:
# Sample document:
# {
#   "custom_id": "89:115",
#   "url": "/v1/embeddings",
#   "method": "POST",
#   "body": {
#     "input": " have been possible without these approaches?",
#     "model": "text-embedding-3-large",
#     "dimensions": 128,
#     "metadata": {
#       "title": "Podcast: Boris Sofman: Waymo, Cozmo, Self-Driving Cars, and the Future of Robotics | Lex Fridman Podcast #241",
#       "podcast_id": "U_AREIyd0Fc",
#       "start_time": 484.52,
#       "stop_time": 487.08
#     }
#   }
# }

# Sample embedding:
# {
#   "id": "batch_req_QZBmHS7FBiVABxcsGiDx2THJ",
#   "custom_id": "89:115",
#   "response": {
#     "status_code": 200,
#     "request_id": "7a55eba082c70aca9e7872d2b694f095",
#     "body": {
#       "object": "list",
#       "data": [
#         {
#           "object": "embedding",
#           "index": 0,
#           "embedding": [
#             0.0035960325,
#             126 more lines....
#             -0.093248844
#           ]
#         }
#       ],
#       "model": "text-embedding-3-large",
#       "usage": {
#         "prompt_tokens": 7,
#         "total_tokens": 7
#       }
#     }
#   },
#   "error": null
# }

In [13]:
import os
import json

dir = "documents/documents"
docs = os.listdir(dir)
podcast = {}
for doc in docs:
  with open(os.path.join(dir, doc)) as input_file:
    for line in input_file:
      data = json.loads(line)
      podcast[data["body"]["metadata"]["podcast_id"]] = data["body"]["metadata"]["title"]

      break;
print(podcast)



{'tueAcSiiqYA': 'Podcast: Jordan Ellenberg: Mathematics of High-Dimensional Shapes and Geometries | Lex Fridman Podcast #190', 'iZRbD7q1n-U': 'Podcast: John Danaher: Grappling, Jiu Jitsu, ADCC, and Animal Combat | Lex Fridman Podcast #328', 'rIAZJNe7YtE': 'Podcast: Eric Weinstein: Geometric Unity and the Call for New Ideas & Institutions | Lex Fridman Podcast #88', 'VeH7qKZr0WI': 'Podcast: Balaji Srinivasan: How to Fix Government, Twitter, Science, and the FDA | Lex Fridman Podcast #331', 'G4hL5Om4IJ4': 'Podcast: Jim Keller: The Future of Computing, AI, Life, and Consciousness | Lex Fridman Podcast #162', 'RvhpncC5jZ8': 'Podcast: John Clarke: The Art of Fighting and the Pursuit of Excellence | Lex Fridman Podcast #143', 'TRdL6ZzWBS0': 'Podcast: Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214', '7Grseeycor4': 'Podcast: Tyler Cowen: Economic Growth & the Fight Against Conformity & Mediocrity | Lex Fridman Podcast #174', 'tOTenjh_8hw': 'Podcast: Bobby L

In [ ]:
# Create table statements that you'll write
#TODO


# may need to run this to enable vector data type if you didn't select AI in service
# CREATE_EXTENSION = "CREATE EXTENSION vector"

# TODO: Add create table statement
CREATE_PODCAST_TABLE = """

"""
# id and title from document - want unique set of titles and podcast id

# TODO: Add create table statement
CREATE_SEGMENT_TABLE = """

"""

conn = psycopg2.connect(CONNECTION)
# TODO: Create tables with psycopg2 (example: https://www.geeksforgeeks.org/executing-sql-query-with-psycopg2-in-python/)


conn.commit()
conn.close()


In [ ]:
## Extract needed data out of JSONL files. This may be the hard part!

# TODO: What data do we need?
# TODO: What data is in the documents jsonl files?
# TODO: What data is in the embedding jsonl files?
# TODO: Get some pandas data frames for our two tables so we can copy the data in!



In [ ]:
#### Optional #####
# In addition to the embedding and document files you might like to load
# the full podcast raw data via the hugging face datasets library

# from datasets import load_dataset
# ds = load_dataset("Whispering-GPT/lex-fridman-podcast")


In [ ]:
# TODO Copy all the "podcast" data into the podcast postgres table!


In [ ]:
# TODO Copy all the "segment" data into the segment postgres table!
# HINT 1: use the recommender.utils.fast_pg_insert function to insert data into the database
# otherwise inserting the 800k documents will take a very, very long time
# HINT 2: if you don't want to use all your memory and crash
# colab, you'll need to either send the data up in chunks
# or write your own function for copying it up. Alternative to chunking maybe start
# with writing it to a CSV and then copy it up?


In [ ]:
## This script is used to query the database
import os
import psycopg2


# Write your queries
# Q1) What are the five most similar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""

""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

In [ ]:
# Q2) What are the five most dissimilar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text, the start time, stop time, and embedding distance


In [ ]:
# Q3) What are the five most similar segments to segment '48:511'

# Input: "Is it is there something especially interesting and profound to you in terms of our current deep learning neural network, artificial neural network approaches and the whatever we do understand about the biological neural network."
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance


In [ ]:
# Q4) What are the five most similar segments to segment '51:56'

# Input: "But what about like the fundamental physics of dark energy? Is there any understanding of what the heck it is?"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance


In [ ]:
# Q5) For each of the following podcast segments, find the five most similar podcast episodes. Hint: You can do this by averaging over the embedding vectors within a podcast episode.

#     a) Segment "267:476"

#     b) Segment '48:511'

#     c) Segment '51:56'

# For each result return the Podcast title and the embedding distance


In [ ]:
# Q6) For podcast episode id = VeH7qKZr0WI, find the five most similar podcast episodes. Hint: you can do a similar averaging procedure as Q5

# Input Episode: "Balaji Srinivasan: How to Fix Government, Twitter, Science, and the FDA | Lex Fridman Podcast #331"
# For each result return the Podcast title and the embedding distance


# Deliverables
You will turn in a ZIP or PDF file containing all your code and a PDF file with the queries and results for questions 1-7.